# Finance Sector Dataset Preparation
**Purpose**: Create a clean, feature-rich dataset for training models on finance sector stocks

**Date**: 2026-02-26

## Overview
This notebook prepares the training dataset for the finance sector:
- 10 tickers: JPM, BAC, GS, MS, WFC, C, BLK, AXP, USB, PNC
- Time period: 2019-2024
- Features: 30+ technical and fundamental indicators
- Target: 5-day forward returns
- Output: Parquet file ready for model training

## 1. Setup

In [ ]:
import sys
from pathlib import Path
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import date, datetime
from tqdm import tqdm

# Add project root to path
project_root = Path.cwd().parent
sys.path.insert(0, str(project_root))

# Import project modules
from src.data.data_loader import DataLoader
from src.feature_engineering.feature_aggregator import FeatureAggregator
from src.feature_engineering.technical_features import TechnicalFeatures
from src.feature_engineering.fundamental_features import FundamentalFeatures

# Set display options
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)
sns.set_style('whitegrid')

print(f"Project root: {project_root}")
print(f"Python version: {sys.version}")

## 2. Configuration

In [ ]:
# Finance sector tickers
FINANCE_TICKERS = ['JPM', 'BAC', 'GS', 'MS', 'WFC', 'C', 'BLK', 'AXP', 'USB', 'PNC']

# Date ranges
START_DATE = date(2019, 1, 1)
END_DATE = date(2024, 12, 31)

TRAIN_END = date(2023, 12, 31)
VAL_START = date(2024, 1, 1)
VAL_END = date(2024, 6, 30)
TEST_START = date(2024, 7, 1)
TEST_END = date(2024, 12, 31)

# Target variable
FORWARD_DAYS = 5  # Predict 5-day forward returns
CLIP_PERCENTILE = 99  # Clip outliers at 99th percentile

# Output paths
OUTPUT_DIR = project_root / 'data' / 'prepared'
OUTPUT_DIR.mkdir(parents=True, exist_ok=True)

print(f"\nFinance Tickers: {len(FINANCE_TICKERS)}")
print(f"Date Range: {START_DATE} to {END_DATE}")
print(f"Target: {FORWARD_DAYS}-day forward returns")
print(f"Output directory: {OUTPUT_DIR}")

## 3. Load Price Data

In [ ]:
# Initialize data loader
data_loader = DataLoader()

# Load price data for all tickers
price_data = {}
metadata = {}

print("Loading price data...\n")
for ticker in tqdm(FINANCE_TICKERS):
    # Load prices
    prices = data_loader.load_stock_prices(ticker, START_DATE, END_DATE)
    
    if not prices.empty:
        price_data[ticker] = prices
        
        # Load metadata
        meta = data_loader.load_company_metadata(ticker)
        if meta:
            metadata[ticker] = meta
        
        print(f"✓ {ticker}: {len(prices)} records from {prices.index[0].date()} to {prices.index[-1].date()}")
    else:
        print(f"✗ {ticker}: No data found")

print(f"\nSuccessfully loaded {len(price_data)} tickers")

## 4. Data Quality Check

In [ ]:
# Analyze data completeness
quality_report = []

for ticker, prices in price_data.items():
    n_records = len(prices)
    date_range = (prices.index[-1] - prices.index[0]).days
    expected_records = date_range * 5 / 7  # Roughly 5 trading days per week
    completeness = (n_records / expected_records) * 100
    
    # Check for missing values
    missing_close = prices['close'].isna().sum()
    missing_volume = prices['volume'].isna().sum()
    
    quality_report.append({
        'ticker': ticker,
        'n_records': n_records,
        'date_range_days': date_range,
        'completeness_pct': completeness,
        'missing_close': missing_close,
        'missing_volume': missing_volume,
        'first_date': prices.index[0].date(),
        'last_date': prices.index[-1].date()
    })

quality_df = pd.DataFrame(quality_report)
print("\nData Quality Report:")
print(quality_df.to_string(index=False))

# Summary statistics
print(f"\n{'='*60}")
print(f"Average completeness: {quality_df['completeness_pct'].mean():.1f}%")
print(f"Min records: {quality_df['n_records'].min()}")
print(f"Max records: {quality_df['n_records'].max()}")
print(f"Total missing close prices: {quality_df['missing_close'].sum()}")

## 5. Feature Engineering

In [ ]:
# Initialize feature aggregator
feature_agg = FeatureAggregator()

# Compute features for all tickers
all_features = []

print("Computing features...\n")
for ticker in tqdm(FINANCE_TICKERS):
    if ticker not in price_data:
        print(f"Skipping {ticker} - no price data")
        continue
    
    prices = price_data[ticker]
    
    # We need to compute features for each date in the dataset
    # Use rolling window approach
    for i in range(100, len(prices)):  # Start after 100 days for indicator warmup
        # Get historical window up to current date
        window_prices = prices.iloc[:i+1]
        current_date = window_prices.index[-1]
        
        # Compute all features using the window
        features = feature_agg.compute_all_features(
            ticker=ticker,
            prices=window_prices,
            sector_performance=None,  # Can add sector rotation features later
            use_cache=True,
            save_cache=True
        )
        
        if features:
            # Add metadata
            features['ticker'] = ticker
            features['date'] = current_date
            features['close'] = window_prices['close'].iloc[-1]
            
            all_features.append(features)

# Convert to DataFrame
features_df = pd.DataFrame(all_features)
print(f"\n✓ Computed features for {len(features_df)} observations")
print(f"✓ Feature columns: {len(features_df.columns)}")
print(f"\nSample features:")
print(features_df.head())

In [ ]:
# Check feature columns
print("\nFeature columns:")
feature_cols = [col for col in features_df.columns if col not in ['ticker', 'date', 'close']]
print(f"Total features: {len(feature_cols)}\n")
for col in sorted(feature_cols):
    print(f"  - {col}")

## 6. Create Target Variable

In [ ]:
# Calculate forward returns for each ticker
print(f"Creating {FORWARD_DAYS}-day forward returns...\n")

features_df = features_df.sort_values(['ticker', 'date'])

# Calculate forward returns
features_df['forward_return'] = np.nan

for ticker in FINANCE_TICKERS:
    ticker_mask = features_df['ticker'] == ticker
    ticker_data = features_df[ticker_mask].copy()
    
    # Calculate forward return
    ticker_data['forward_return'] = ticker_data['close'].shift(-FORWARD_DAYS) / ticker_data['close'] - 1
    
    # Update main dataframe
    features_df.loc[ticker_mask, 'forward_return'] = ticker_data['forward_return'].values

# Remove rows without forward returns (last N days of each ticker)
before_drop = len(features_df)
features_df = features_df.dropna(subset=['forward_return'])
after_drop = len(features_df)

print(f"✓ Created forward returns")
print(f"✓ Dropped {before_drop - after_drop} rows without forward returns")
print(f"✓ Remaining observations: {after_drop}")

# Handle outliers (clip at percentiles)
lower = features_df['forward_return'].quantile((100 - CLIP_PERCENTILE) / 100)
upper = features_df['forward_return'].quantile(CLIP_PERCENTILE / 100)

print(f"\nReturn statistics (before clipping):")
print(features_df['forward_return'].describe())
print(f"\nClipping outliers at {CLIP_PERCENTILE}th percentile: [{lower:.4f}, {upper:.4f}]")

features_df['forward_return'] = features_df['forward_return'].clip(lower, upper)

print(f"\nReturn statistics (after clipping):")
print(features_df['forward_return'].describe())

In [ ]:
# Visualize return distribution
plt.figure(figsize=(12, 5))

plt.subplot(1, 2, 1)
features_df['forward_return'].hist(bins=50, edgecolor='black')
plt.xlabel(f'{FORWARD_DAYS}-Day Forward Return')
plt.ylabel('Frequency')
plt.title('Distribution of Forward Returns')
plt.axvline(0, color='red', linestyle='--', linewidth=2, label='Zero')
plt.legend()

plt.subplot(1, 2, 2)
for ticker in FINANCE_TICKERS:
    ticker_returns = features_df[features_df['ticker'] == ticker]['forward_return']
    plt.plot(ticker_returns.index, ticker_returns.cumsum(), label=ticker, alpha=0.7)
plt.xlabel('Observation')
plt.ylabel('Cumulative Return')
plt.title('Cumulative Returns by Ticker')
plt.legend(bbox_to_anchor=(1.05, 1), loc='upper left')

plt.tight_layout()
plt.show()

## 7. Handle Missing Values

In [ ]:
# Check for missing values in features
missing_counts = features_df.isnull().sum()
missing_features = missing_counts[missing_counts > 0].sort_values(ascending=False)

if len(missing_features) > 0:
    print("Features with missing values:\n")
    for feature, count in missing_features.items():
        pct = (count / len(features_df)) * 100
        print(f"  {feature}: {count} ({pct:.2f}%)")
    
    # Strategy: Fill missing values with median for numerical features
    print("\nFilling missing values with median...")
    for col in missing_features.index:
        if col not in ['ticker', 'date']:
            median_val = features_df[col].median()
            features_df[col] = features_df[col].fillna(median_val)
    
    print("✓ Missing values handled")
else:
    print("✓ No missing values in features")

# Final check
print(f"\nFinal dataset shape: {features_df.shape}")
print(f"Remaining missing values: {features_df.isnull().sum().sum()}")

## 8. Train/Validation/Test Split

In [ ]:
# Convert date to datetime if needed
if not isinstance(features_df['date'].iloc[0], pd.Timestamp):
    features_df['date'] = pd.to_datetime(features_df['date'])

# Split by date
train_df = features_df[features_df['date'] <= pd.Timestamp(TRAIN_END)].copy()
val_df = features_df[
    (features_df['date'] >= pd.Timestamp(VAL_START)) & 
    (features_df['date'] <= pd.Timestamp(VAL_END))
].copy()
test_df = features_df[
    (features_df['date'] >= pd.Timestamp(TEST_START)) & 
    (features_df['date'] <= pd.Timestamp(TEST_END))
].copy()

print("Dataset splits:")
print(f"\nTraining set:")
print(f"  Dates: {train_df['date'].min().date()} to {train_df['date'].max().date()}")
print(f"  Observations: {len(train_df)}")
print(f"  Tickers: {train_df['ticker'].nunique()}")

print(f"\nValidation set:")
print(f"  Dates: {val_df['date'].min().date()} to {val_df['date'].max().date()}")
print(f"  Observations: {len(val_df)}")
print(f"  Tickers: {val_df['ticker'].nunique()}")

print(f"\nTest set:")
print(f"  Dates: {test_df['date'].min().date()} to {test_df['date'].max().date()}")
print(f"  Observations: {len(test_df)}")
print(f"  Tickers: {test_df['ticker'].nunique()}")

print(f"\nTotal: {len(train_df) + len(val_df) + len(test_df)} observations")

In [ ]:
# Visualize split
plt.figure(figsize=(14, 6))

for ticker in FINANCE_TICKERS[:5]:  # Show first 5 tickers for clarity
    ticker_data = features_df[features_df['ticker'] == ticker]
    plt.scatter(ticker_data[ticker_data['date'] <= pd.Timestamp(TRAIN_END)]['date'], 
                [ticker] * len(ticker_data[ticker_data['date'] <= pd.Timestamp(TRAIN_END)]),
                alpha=0.3, s=10, color='blue', label='Train' if ticker == FINANCE_TICKERS[0] else '')
    plt.scatter(ticker_data[(ticker_data['date'] >= pd.Timestamp(VAL_START)) & 
                             (ticker_data['date'] <= pd.Timestamp(VAL_END))]['date'],
                [ticker] * len(ticker_data[(ticker_data['date'] >= pd.Timestamp(VAL_START)) & 
                                            (ticker_data['date'] <= pd.Timestamp(VAL_END))]),
                alpha=0.3, s=10, color='orange', label='Val' if ticker == FINANCE_TICKERS[0] else '')
    plt.scatter(ticker_data[(ticker_data['date'] >= pd.Timestamp(TEST_START)) & 
                             (ticker_data['date'] <= pd.Timestamp(TEST_END))]['date'],
                [ticker] * len(ticker_data[(ticker_data['date'] >= pd.Timestamp(TEST_START)) & 
                                            (ticker_data['date'] <= pd.Timestamp(TEST_END))]),
                alpha=0.3, s=10, color='green', label='Test' if ticker == FINANCE_TICKERS[0] else '')

plt.xlabel('Date')
plt.ylabel('Ticker')
plt.title('Train/Validation/Test Split Visualization')
plt.legend()
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

## 9. Save Dataset

In [ ]:
# Save datasets to parquet (efficient format)
train_path = OUTPUT_DIR / 'finance_train.parquet'
val_path = OUTPUT_DIR / 'finance_val.parquet'
test_path = OUTPUT_DIR / 'finance_test.parquet'
full_path = OUTPUT_DIR / 'finance_full.parquet'

print("Saving datasets...\n")

train_df.to_parquet(train_path, index=False)
print(f"✓ Training set saved: {train_path}")
print(f"  Size: {train_path.stat().st_size / 1024 / 1024:.2f} MB")

val_df.to_parquet(val_path, index=False)
print(f"✓ Validation set saved: {val_path}")
print(f"  Size: {val_path.stat().st_size / 1024 / 1024:.2f} MB")

test_df.to_parquet(test_path, index=False)
print(f"✓ Test set saved: {test_path}")
print(f"  Size: {test_path.stat().st_size / 1024 / 1024:.2f} MB")

features_df.to_parquet(full_path, index=False)
print(f"✓ Full dataset saved: {full_path}")
print(f"  Size: {full_path.stat().st_size / 1024 / 1024:.2f} MB")

In [ ]:
# Save metadata
import json

metadata_dict = {
    'creation_date': datetime.now().isoformat(),
    'sector': 'finance',
    'tickers': FINANCE_TICKERS,
    'n_tickers': len(FINANCE_TICKERS),
    'date_range': {
        'start': START_DATE.isoformat(),
        'end': END_DATE.isoformat()
    },
    'splits': {
        'train': {
            'end_date': TRAIN_END.isoformat(),
            'n_observations': len(train_df)
        },
        'validation': {
            'start_date': VAL_START.isoformat(),
            'end_date': VAL_END.isoformat(),
            'n_observations': len(val_df)
        },
        'test': {
            'start_date': TEST_START.isoformat(),
            'end_date': TEST_END.isoformat(),
            'n_observations': len(test_df)
        }
    },
    'target': {
        'name': 'forward_return',
        'forward_days': FORWARD_DAYS,
        'clip_percentile': CLIP_PERCENTILE
    },
    'features': {
        'n_features': len(feature_cols),
        'feature_names': feature_cols
    },
    'statistics': {
        'total_observations': len(features_df),
        'train_observations': len(train_df),
        'val_observations': len(val_df),
        'test_observations': len(test_df)
    }
}

metadata_path = OUTPUT_DIR / 'finance_dataset_metadata.json'
with open(metadata_path, 'w') as f:
    json.dump(metadata_dict, f, indent=2)

print(f"\n✓ Metadata saved: {metadata_path}")

## 10. Dataset Summary

In [ ]:
print("="*70)
print("FINANCE SECTOR DATASET SUMMARY")
print("="*70)

print(f"\n📊 Dataset Statistics:")
print(f"  Total observations: {len(features_df):,}")
print(f"  Training: {len(train_df):,} ({len(train_df)/len(features_df)*100:.1f}%)")
print(f"  Validation: {len(val_df):,} ({len(val_df)/len(features_df)*100:.1f}%)")
print(f"  Test: {len(test_df):,} ({len(test_df)/len(features_df)*100:.1f}%)")

print(f"\n🏦 Tickers: {len(FINANCE_TICKERS)}")
print(f"  {', '.join(FINANCE_TICKERS)}")

print(f"\n📅 Date Range:")
print(f"  Overall: {START_DATE} to {END_DATE}")
print(f"  Training: up to {TRAIN_END}")
print(f"  Validation: {VAL_START} to {VAL_END}")
print(f"  Test: {TEST_START} to {TEST_END}")

print(f"\n🎯 Target Variable:")
print(f"  Name: forward_return")
print(f"  Period: {FORWARD_DAYS} days")
print(f"  Mean: {features_df['forward_return'].mean():.4f}")
print(f"  Std: {features_df['forward_return'].std():.4f}")
print(f"  Min: {features_df['forward_return'].min():.4f}")
print(f"  Max: {features_df['forward_return'].max():.4f}")

print(f"\n🔧 Features: {len(feature_cols)}")
print(f"  Technical indicators: ~17")
print(f"  Fundamental features: ~15+")

print(f"\n💾 Output Files:")
print(f"  {train_path.name}")
print(f"  {val_path.name}")
print(f"  {test_path.name}")
print(f"  {full_path.name}")
print(f"  {metadata_path.name}")

print(f"\n✅ Dataset preparation complete!")
print(f"\nNext steps:")
print(f"  1. Train linear model: Create 03_train_linear_model.ipynb")
print(f"  2. Load dataset: pd.read_parquet('{train_path}')")
print(f"  3. Train and evaluate models")
print("="*70)